In [1]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "05_create_dataset/05_split_tfrecord.ipynb"
_nb_title = "Splitting dataset and writing TF Records"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
_nb_safetitle = _nb_title.replace(' ', '+')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_safetitle, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Splitting+dataset+and+writing+TF+Records&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F05_create_dataset%2F05_split_tfrecord.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F05_create_dataset%2F05_split_tfrecord.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/05_create_dataset/05_split_tfrecord.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/05_create_dataset/05_split_tfrecord.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/05_create_dataset/05_split_tfrecord.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# Splitting dataset and writing TF Records

This notebook shows you how to split a dataset into training, validation, testing and write those images into TensorFlow Record files.


In [1]:
import pandas as pd
df = pd.read_csv('gs://cloud-ml-data/img/flower_photos/all_data.csv', names=['image','label'])
df.head()

,image,label
0,gs://cloud-ml-data/img/flower_photos/daisy/100...,daisy
1,gs://cloud-ml-data/img/flower_photos/daisy/101...,daisy
2,gs://cloud-ml-data/img/flower_photos/daisy/101...,daisy
3,gs://cloud-ml-data/img/flower_photos/daisy/101...,daisy
4,gs://cloud-ml-data/img/flower_photos/daisy/101...,daisy


In [2]:
import numpy as np
np.random.seed(10)
rnd = np.random.rand(len(df))
train = df[ rnd < 0.8  ]
valid = df[ (rnd >= 0.8) & (rnd < 0.9) ]
test  = df[ rnd >= 0.9 ]
print(len(df), len(train), len(valid), len(test))

3670 2930 359 381


In [3]:
%%bash
rm -rf output
mkdir output

In [4]:
train.to_csv('output/train.csv', header=False, index=False)
valid.to_csv('output/valid.csv', header=False, index=False)
test.to_csv('output/test.csv', header=False, index=False)

In [5]:
!head output/test.csv

gs://cloud-ml-data/img/flower_photos/daisy/10466290366_cc72e33532.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10712722853_5632165b04.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/11642632_1e7627a2cc.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/13583238844_573df2de8e_m.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/1374193928_a52320eafa.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/13953307149_f8de6a768c_m.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/14471433500_cdaa22e3ea_m.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/14523675369_97c31d0b5b.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/163978992_8128b49d3e_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/16401288243_36112bd52f_m.jpg,daisy


## Writing TF Records using Apache Beam

For speed, we'll illustrate writing just 5 records

In [6]:
outdf = test.head()
len(outdf)

5

In [7]:
outdf.values

array([['gs://cloud-ml-data/img/flower_photos/daisy/10466290366_cc72e33532.jpg',
        'daisy'],
       ['gs://cloud-ml-data/img/flower_photos/daisy/10712722853_5632165b04.jpg',
        'daisy'],
       ['gs://cloud-ml-data/img/flower_photos/daisy/11642632_1e7627a2cc.jpg',
        'daisy'],
       ['gs://cloud-ml-data/img/flower_photos/daisy/13583238844_573df2de8e_m.jpg',
        'daisy'],
       ['gs://cloud-ml-data/img/flower_photos/daisy/1374193928_a52320eafa.jpg',
        'daisy']], dtype=object)

In [8]:
!gsutil cat gs://cloud-ml-data/img/flower_photos/dict.txt

daisy
dandelion
roses
sunflowers
tulips


In [9]:
import tensorflow as tf
with tf.io.gfile.GFile('gs://cloud-ml-data/img/flower_photos/dict.txt', 'r') as f:
    LABELS = [line.rstrip() for line in f]
print('Read in {} labels, from {} to {}'.format(
    len(LABELS), LABELS[0], LABELS[-1]))

Read in 5 labels, from daisy to tulips


In [22]:
import apache_beam as beam
import tensorflow as tf

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_array_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def read_and_decode(filename):
    IMG_HEIGHT = 224
    IMG_WIDTH = 224
    IMG_CHANNELS = 3
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.reshape(tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]), [-1])

def create_tfrecord(filename, label, label_int):
    img = read_and_decode(filename)
    return tf.train.Example(features=tf.train.Features(feature={
        'image': _float_array_feature(img),
        'label': _bytes_feature(label.encode('utf-8')),
        'label_int': _int64_feature(label_int)
    })).SerializeToString()

with beam.Pipeline() as p:
    (p 
     | 'input_df' >> beam.Create(outdf.values)
     | 'create_tfrecord' >> beam.Map(lambda x: create_tfrecord(x[0], x[1], LABELS.index(x[1])))
     | 'write' >> beam.io.tfrecordio.WriteToTFRecord('output/train')
    )

In [18]:
!ls -l output/train-*

-rw-r--r-- 1 jupyter jupyter 3010975 Dec 24 20:48 output/train-00000-of-00001


## Running on Dataflow

Apache Beam code can be executed in a serverless way using Cloud Dataflow.

The key thing is to:
<br/>
Replace beam.Pipeline() by:

<pre>
options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': JOBNAME,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'save_main_session': True
  }
opts = beam.pipeline.PipelineOptions(flags=[], **options)
with beam.Pipeline(RUNNER, options=opts) as p:
</pre>

In [ ]:
%%bash
PROJECT=$(gcloud config get-value project)
BUCKET=${PROJECT}

python ./jpeg_to_tfrecord.py \
       --all_data gs://cloud-ml-data/img/flower_photos/all_data.csv \
       --labels_file gs://cloud-ml-data/img/flower_photos/dict.txt \
       --project_id $PROJECT \
       --output_dir gs://${BUCKET}/data/flower_tfrecords

<img src="dataflow_pipeline.png" />

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.